# 5.1自定义块

In [145]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0105, -0.0579, -0.0360, -0.0529,  0.1701, -0.1119, -0.0848,  0.0012,
         -0.0375,  0.2609],
        [-0.0386, -0.1635, -0.0158,  0.0186,  0.1409, -0.0974, -0.1133,  0.0048,
          0.0135,  0.1996]], grad_fn=<AddmmBackward0>)

In [146]:
import torch
from torch import nn
from torch.nn import functional as F

class MLP(nn.Module):
    #用模型参数声明层，声明两个全连接的层
    # 调用MLP的父类Module的构造函数来执行必要的初始化。
    # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256) #隐藏层
        self.out = nn.Linear(256,10)  #输出层
    
    #定义模型的前向传播
    def forward(self , X):
        return self.out(F.relu(self.hidden(X)))
    #前向传播函数，它以X作为输入， 计算带有激活函数的隐藏表示，并输出其未规范化的输出值。 在这个MLP实现中，两个层都是实例变量。

In [147]:
net = MLP()
X = torch.rand(2, 20)
net(X)

tensor([[ 0.0828, -0.0236,  0.0535, -0.1046, -0.0015,  0.0157, -0.3442,  0.0924,
          0.1537, -0.1105],
        [ 0.0544,  0.0970, -0.0089,  0.0646,  0.0658,  0.0156, -0.4096,  0.1470,
          0.2030, -0.2075]], grad_fn=<AddmmBackward0>)

## 顺序块

为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

一种将块逐个追加到列表中的函数；

一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [148]:
class MySequential(nn.Module):
    #__init__函数将每个模块逐个添加到有序字典_modules中。 
    def __init__(self,*args):
        super().__init__()
        for idx , module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module
    
    def forward(self,X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

__init__函数将每个模块逐个添加到有序字典_modules中。每个Module都有一个_modules属性.我们使用它而不是自己定义一个Python列表.简而言之，_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。
当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。 现在可以使用我们的MySequential类重新实现多层感知机。

In [149]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)

tensor([[-0.0852, -0.1986,  0.0064,  0.0438, -0.2086, -0.1248, -0.0972, -0.1183,
         -0.2022, -0.0375],
        [-0.0086, -0.0715, -0.0217, -0.0202, -0.1706, -0.0803,  0.0451, -0.0731,
         -0.1551,  0.0143]], grad_fn=<AddmmBackward0>)

## 在前向传播函数中执行代码

Sequential类使模型构造变得简单， 允许我们组合新的架构，而不必定义自己的类。 然而，并不是所有的架构都是简单的顺序架构。 当需要更强的灵活性时，我们需要定义自己的块。 例如，我们可能希望在前向传播函数中执行Python的控制流。 此外，我们可能希望执行任意的数学运算， 而不是简单地依赖预定义的神经网络层。

In [150]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        #不计算梯度的随机权重参数。因此其在训练期间保持不变
        super().__init__()
        self.rand_weight = torch.rand((20,20) , requires_grad=False)
        self.linear = nn.Linear(20,20)

    def forward(self, X):
        X = self.linear(X)
        #使用创建的常量参数以及relu和mm函数(矩阵乘法)
        X = F.relu(torch.mm(X,self.rand_weight) + 1)
        #复用全连接层。相当于两个全连接层共享参数
        X = self.linear(X)
        #控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情： 它运行了一个while循环，在L1范数大于1的条件下， 将输出向量除以2，直到它满足条件为止。 最后，模型返回了X中所有项的和。 注意，此操作可能不会常用于在任何实际任务中， 我们只展示如何将任意代码集成到神经网络计算的流程中。

In [151]:
net = FixedHiddenMLP()
net(X)

tensor(0.2284, grad_fn=<SumBackward0>)

## 混合搭配各个块

我们可以混合搭配各种组合块的方法。 在下面的例子中，我们以一些想到的方法嵌套块。

In [152]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.1434, grad_fn=<SumBackward0>)

# 5.2参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。 此时，我们的目标是找到使损失函数最小化的模型参数值。 经过训练后，我们将需要使用这些参数来做出未来的预测。 此外，有时我们希望提取参数，以便在其他环境中复用它们， 将模型保存下来，以便它可以在其他软件中执行， 或者为了获得科学的理解而进行检查。

如下是具有单隐藏层的多层感知机。

In [153]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8) , nn.ReLU() , nn.Linear( 8,1 ))
X = torch.rand(size = (2,4))
net(X)

tensor([[0.5294],
        [0.4627]], grad_fn=<AddmmBackward0>)

## 参数访问

我们从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 这就像模型是一个列表一样，每层的参数都在其属性中。 如下所示，我们可以检查第二个全连接层的参数。

In [154]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0085,  0.1868,  0.1207, -0.3504,  0.0742,  0.3194,  0.1206,  0.1577]])), ('bias', tensor([0.1518]))])


输出的结果告诉我们： 首先，这个全连接层包含两个参数，分别是该层的权重和偏置。 两者都存储为单精度浮点数（float32）。 注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### 目标参数

每个参数都表示为参数类的一个实例。 要对参数执行任何操作，首先我们需要访问底层的数值。 有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。 下面的代码从第二个全连接层（即第三个神经网络层）提取偏置， 提取后返回的是一个参数类实例，并进一步访问该参数的值。

In [155]:
print('----------')
print(type(net[2].bias))
print('----------')
print(net[2].bias)
print('----------')
print(net[2].bias.data)
print('----------')

----------
<class 'torch.nn.parameter.Parameter'>
----------
Parameter containing:
tensor([0.1518], requires_grad=True)
----------
tensor([0.1518])
----------


参数是复合的对象，包含值、梯度和额外信息。 这就是我们需要显式参数值的原因。 除了值之外，我们还可以访问每个参数的梯度。 在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。

In [156]:
net[2].weight.grad == None

True

### 一次性访问所有参数

需要对所有参数执行操作时，逐个访问它们可能会很麻烦。 当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂， 因为我们需要递归整个树来提取每个子块的参数。 下面通过演示来比较访问第一个全连接层的参数和访问所有层。

In [157]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


另一种访问网络参数的方式，如下所示。pytorch 中的 state_dict 是一个简单的python的字典对象,将每一层与它的对应参数建立映射关系.(如model的每一层的weights及偏置等等)

In [158]:
net.state_dict()['2.bias'].data

tensor([0.1518])

总结：model.state_dict()、model.parameters()、model.named_parameters()这三个方法都可以查看Module的参数信息，用于更新参数，或者用于模型的保存。

### 从嵌套块收集参数

In [159]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(), 
                         nn.Linear(8,4),nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        #开始嵌套
        net.add_module(f'block{i}',block1())
    return net

rgnet = nn.Sequential(block2() , nn.Linear(4,1))
X = torch.rand(size=(2,4))
rgnet(X)

tensor([[0.1786],
        [0.1786]], grad_fn=<AddmmBackward0>)

In [160]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。 下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。

In [161]:
rgnet[0][1][0].bias.data   #第一个主要的块中、第二个子块的第一层的偏置项

tensor([ 0.4626,  0.4713, -0.3774,  0.1641, -0.1860, -0.3316, -0.0127,  0.1414])

## 参数初始化

 深度学习框架提供默认随机初始化， 也允许我们创建自定义初始化方法， 满足我们通过其他规则实现初始化权重。

### 内置初始化

首先调用内置的初始化器。 下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量， 且将偏置参数设置为0。

In [162]:
def init_normal(m):
    if type(m)==nn.Linear:
        nn.init.normal_(m.weight , mean = 0 , std = 0.01)  #正态分布
        nn.init.zeros_(m.bias)

net.apply(init_normal)
net[0].weight.data[0] , net[0].bias.data[0]

(tensor([ 0.0091,  0.0062,  0.0053, -0.0095]), tensor(0.))

还可以将所有参数初始化为给定的常数，比如初始化为1。

In [163]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)  #初始化为给定常数
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以对某些块应用不同的初始化方法。 例如，下面我们使用Xavier初始化方法初始化第一个神经网络层， 然后将第三个神经网络层初始化为常量值42。

In [164]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)

def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight , 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.2578, -0.4697,  0.2993, -0.4544])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### 自定义初始化

w~U(5,10) Prob=0.25;

w=0 Prob=0.5

w~U(-10,-5) Prob=0.25

In [165]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init" , *[(name,param.shape) for name ,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight , -10 ,10) 
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  8.2252,  0.0000, -0.0000],
        [-7.6592, -7.3616,  8.5523, -0.0000]], grad_fn=<SliceBackward0>)

还有，我们始终可以直接设置参数。

In [166]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  9.2252,  1.0000,  1.0000])

## 参数绑定

有时我们希望在多个层间共享参数： 我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。

In [167]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8) , nn.ReLU(),
                    shared , nn.ReLU(),
                    shared,nn.ReLU(),
                    nn.Linear(8,1))

net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第2个和第4个神经网络层的参数是绑定的。 它们不仅值相等，而且由相同的张量表示。 因此，如果我们改变其中一个参数，另一个参数也会改变。 

答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层 （即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。

共享参数通常可以节省内存，并在以下方面具有特定的好处：

1、对于图像识别中的CNN，共享参数使网络能够在图像中的任何地方而不是仅在某个区域中查找给定的功能。

2、对于RNN，它在序列的各个时间步之间共享参数，因此可以很好地推广到不同序列长度的示例。

3、对于自动编码器，编码器和解码器共享参数。 在具有线性激活的单层自动编码器中，共享权重会在权重矩阵的不同隐藏层之间强制正交。

# 5.3延后初始化

深度学习框架无法判断网络的输入维度是什么。 这里的诀窍是框架的延后初始化（defers initialization）， 即直到数据第一次通过模型传递时，框架才会动态地推断出每个层的大小。

当使用卷积神经网络时， 由于输入维度（即图像的分辨率）将影响每个后续层的维数， 有了该技术将更加方便。 现在我们在编写代码时无须知道维度是什么就可以设置参数， 这种能力可以大大简化定义和修改模型的任务。

In [168]:
import torch
from torch import nn
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(),nn.Linear(256,10))
#可见nn.LazyLinear(256)没有指定输入维度
print(net[0].weight)#尚未初始化
print(net)

<UninitializedParameter>
Sequential(
  (0): LazyLinear(in_features=0, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [169]:
[net[i].state_dict() for i in range(len(net))]

[OrderedDict([('weight', <UninitializedParameter>),
              ('bias', <UninitializedParameter>)]),
 OrderedDict(),
 OrderedDict([('weight',
               tensor([[ 0.0270, -0.0128, -0.0246,  ...,  0.0613,  0.0121, -0.0520],
                       [ 0.0491,  0.0268,  0.0408,  ..., -0.0109, -0.0179,  0.0507],
                       [ 0.0287, -0.0124, -0.0110,  ...,  0.0131, -0.0165, -0.0114],
                       ...,
                       [-0.0283, -0.0152, -0.0542,  ...,  0.0555,  0.0489, -0.0183],
                       [ 0.0212,  0.0262, -0.0621,  ..., -0.0258, -0.0344,  0.0078],
                       [-0.0125, -0.0466,  0.0130,  ...,  0.0126,  0.0561, -0.0024]])),
              ('bias',
               tensor([ 0.0599,  0.0558,  0.0434, -0.0091,  0.0352, -0.0075, -0.0202,  0.0617,
                       -0.0408,  0.0091]))])]

可见输入层的参数是未初始化的

In [170]:
low = torch.finfo(torch.float32).min/10
high = torch.finfo(torch.float32).max/10
X = torch.zeros([2,20],dtype=torch.float32).uniform_(low, high)
net(X)
print(net)

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


# 5.4自定义层

我们可以用创造性的方式组合不同的层，从而设计出适用于各种任务的架构。 例如，研究人员发明了专门用于处理图像、文本、序列数据和执行动态规划的层。 有时我们会遇到或要自己发明一个现在在深度学习框架中还不存在的层。 在这些情况下，必须构建自定义层。

## 不带参数的层

构造一个没有任何参数的自定义层。 下面的CenteredLayer类要从其输入中减去均值。 要构建它，我们只需继承基础层类并实现前向传播功能。

In [171]:
import torch
import torch.nn.functional as F
import torch.nn as nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,X):
        return X - X.mean()

In [172]:
layer = CenteredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

现在，我们可以将层作为组件合并到更复杂的模型中。

In [173]:
net = nn.Sequential(nn.Linear(8,128),CenteredLayer())

In [174]:
Y = net(torch.rand(4,8))
Y.mean()

tensor(3.2596e-09, grad_fn=<MeanBackward0>)

## 带参数的层

下面我们继续定义具有参数的层， 这些参数可以通过训练进行调整。 我们可以使用内置函数来创建参数，这些函数提供一些基本的管理功能。 比如管理访问、初始化、共享、保存和加载模型参数。 这样做的好处之一是：我们不需要为每个自定义层编写自定义的序列化程序。

实现自定义版本的全连接层。 该层需要两个参数，一个用于表示权重，另一个用于表示偏置项。 在此实现中，我们使用修正线性单元作为激活函数。 该层需要输入参数：in_units和units，分别表示输入数和输出数。

In [175]:
class MyLinear(nn.Module):
    def __init__(self , in_units , units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units , units))
        self.bias = nn.Parameter(torch.randn(units , ))

    def forward(self , X):
        linear = torch.matmul(X , self.weight.data) + self.bias.data
        return F.relu(linear)

In [176]:
linear = MyLinear(5,3)
linear.weight

Parameter containing:
tensor([[ 1.0988,  0.5752, -0.7699],
        [ 1.3154, -0.0306, -1.1572],
        [ 0.5853,  1.0681, -0.5981],
        [-0.0198,  2.7503,  0.1402],
        [ 0.0275, -0.1438,  0.0364]], requires_grad=True)

我们可以使用自定义层直接执行前向传播计算。

In [177]:
linear(torch.rand(2,5))

tensor([[1.3671, 3.2402, 0.0000],
        [1.7066, 1.8952, 0.0000]])

In [178]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

我们可以通过基本层类设计自定义层。这允许我们定义灵活的新层，其行为与深度学习框架中的任何现有层不同。

在自定义层定义完成后，我们就可以在任意环境和网络架构中调用该自定义层。

层可以有局部参数，这些参数可以通过内置函数创建。

# 5.5读写文件

有时我们希望保存训练的模型， 以备将来在各种环境中使用（比如在部署中进行预测）。 此外，当运行一个耗时较长的训练过程时， 最佳的做法是定期保存中间结果， 以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。

## 加载和保存张量

对于单个张量，我们可以直接调用load和save函数分别读写它们。 这两个函数都要求我们提供一个名称，save要求将要保存的变量作为输入。

In [179]:
import torch 
import torch.nn.functional as F
from torch import nn

x = torch.arange(4)
torch.save(x , 'x-file')

我们现在可以将存储在文件中的数据读回内存。

In [180]:
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

我们可以存储一个张量列表，然后把它们读回内存。

In [181]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

我们甚至可以写入或读取从字符串映射到张量的字典。 当我们要读取或写入模型中的所有权重时，这很方便。

In [182]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

## 加载和保存模型参数

保存单个权重向量（或其他张量）确实有用， 但是如果我们想保存整个模型，并在以后加载它们， 单独保存每个向量则会变得很麻烦。 

save和load函数可用于张量对象的文件读写。

我们可以通过参数字典保存和加载网络的全部参数。

保存架构必须在代码中完成，而不是在参数中完成。



深度学习框架提供了内置函数来保存和加载整个网络。 需要注意的一个重要细节是，这将保存模型的参数而不是保存整个模型。 例如，如果我们有一个3层多层感知机，我们需要单独指定架构。 因为模型本身可以包含任意代码，所以模型本身难以序列化。 因此，为了恢复模型，我们需要用代码生成架构， 然后从磁盘加载参数。

In [183]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20,256)
        self.output = nn.Linear(256,10)
    
    def forward(self , X):
        return self.output(F.relu(self.hidden(X)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

我们将模型的参数存储在一个叫做“mlp.params”的文件中。

In [184]:
torch.save(net.state_dict(), 'mlp.params')

为了恢复模型，我们实例化了原始多层感知机模型的一个备份。 这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数。

In [185]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

由于两个实例具有相同的模型参数，在输入相同的X时， 两个实例的计算结果应该相同。 让我们来验证一下。

In [186]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])

# 5.6 GPU

## 计算设备

我们可以指定用于存储和计算的设备，如CPU和GPU。 默认情况下，张量是在内存中创建的，然后使用CPU计算它。

### Nvidia

在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。 应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。 然而，gpu设备只代表一个卡和相应的显存。 如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第i块GPU（i从0开始）。 另外，cuda:0和cuda是等价的。

### Apple Silicone

在使用Apple Silicone的系统中，CPU和GPU可以用torch.device('cpu') 和torch.device('mps')表示。

In [187]:
import torch
from torch import nn

torch.device('cpu'), torch.device('mps')
# torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='mps'))

我们可以查询可用gpu的数量。（仅在Nvidia的机器上有效）

In [188]:
torch.cuda.device_count()

0

现在我们定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [189]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

## 张量与GPU

我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

In [190]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

需要注意的是，无论何时我们要对多个项进行操作， 它们都必须在同一个设备上。 例如，如果我们对两个张量求和， 我们需要确保两个张量都位于同一个设备上， 否则框架将不知道在哪里存储结果，甚至不知道在哪里执行计算。

### 存储在GPU上

有几种方法可以在GPU上存储张量。 例如，我们可以在创建张量时指定存储设备。接 下来，我们在第一个gpu上创建张量变量X。 在GPU上创建的张量只消耗这个GPU的显存。 我们可以使用nvidia-smi命令查看显存使用情况。 一般来说，我们需要确保不创建超过GPU显存限制的数据。

In [191]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]])

假设我们至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量。

In [192]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.3800, 0.3763, 0.6947],
        [0.7487, 0.8722, 0.6317]])

### 复制

这部分内容看《动手学深度学习》第5.6小节